This notebook contains a few examples for bcc Fe to illustrate how VASP_job works

- Note: They only show how to create the inputs (set_calculation()), to submit the job, use:
   
   self.run_vasp()

In [1]:
from subprocess import run
import numpy as np
from VASP_job.code.main import VASP_job

import pandas as pd
import ase
from ase.spacegroup import crystal

# Define a custom formatting function for vectors in pandas
format_vector = lambda vector: '[' + ', '.join(['{:.10f}'.format(x) for x in vector]) + ']'

In [2]:
# set executable
executable_path= "/home/emendive/workbench/work/from_claix/codes/VASP/vasp.5.4.4-flag4/bin"
potential_path = "/home/emendive/workbench/work/from_claix/codes/VASP/potentials/potpaw_PBE"

# set calculation

set crystal structure

In [3]:
a = 3.95
alpha = 90
structure = crystal(('Ga', 'Mn', 'N'),
                       basis=[(0.00, 0.00, 0.00), (0.00, 0.50, 0.50), (0.50, 0.50, 0.50)],
                       spacegroup=221,
                       cellpar=[a, a, a, alpha, alpha, alpha])

set calculation

In [4]:
# create vasp class
vasp = VASP_job(verbose="low", executable_path=executable_path, potential_path=potential_path)

In [5]:
# set magdirs for a triangular AFM state
value_of_m = 0.99

Dtheta = 120 * np.pi/180
theta = -Dtheta

number_of_atoms = len(structure)
vasp.io.magnetic_inputs.magdirs   = []
vasp.io.magnetic_inputs.ms    = []
# vasp.io.magnetic_inputs.B_CONSTRs = []
for i in range(number_of_atoms):
   if 'Mn' == structure.get_chemical_symbols()[i]:
      theta += Dtheta
      vasp.io.magnetic_inputs.magdirs.append([np.cos(theta), np.sin(theta), 0])
      vasp.io.magnetic_inputs.ms.append(value_of_m)
      # vasp.io.magnetic_inputs.B_CONSTRs.append([0.0, 0.0, 0.0])
   else:
      vasp.io.magnetic_inputs.magdirs.append([0.0, 0.0, 0.0])
      vasp.io.magnetic_inputs.ms.append(False)
      # vasp.io.magnetic_inputs.B_CONSTRs.append([0.0, 0.0, 0.0])

In [6]:
# first prepare bfields (constraints)
vasp.prepare_bfields(I_CONSTRAINED="4", LAMBDA="1")
# set calculation, which creates all input files
vasp.set_calculation(structure, vasp.io.magnetic_inputs)

visualize some inputs

In [7]:
vasp.df

,elements,positions,magdirs,ms,betahs,magmoms,B_CONSTRs
0,Ga,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]",False,inf,"[0.0, 0.0, 0.0]","[0, 0, 0]"
1,Mn,"[0.0, 1.975, 1.975]","[1.0, 0.0, 0]",0.99,100.0,"[0.993267716602226, 0.031166234427542957, 0.11...","[0, 0, 0]"
2,Mn,"[1.975, 0.0, 1.975]","[-0.4999999999999998, 0.8660254037844387, 0]",0.99,100.0,"[-0.43249701088604814, 0.9016317304447093, -0....","[0, 0, 0]"
3,Mn,"[1.975, 1.975, 0.0]","[-0.5000000000000004, -0.8660254037844384, 0]",0.99,100.0,"[-0.4852052526560532, -0.8620147316268041, -0....","[0, 0, 0]"
4,N,"[1.975, 1.975, 1.975]","[0.0, 0.0, 0.0]",False,inf,"[0.0, 0.0, 0.0]","[0, 0, 0]"


In [8]:
df_magmoms = vasp.df["magmoms"].apply(format_vector)
df_magmoms

0       [0.0000000000, 0.0000000000, 0.0000000000]
1       [0.9932677166, 0.0311662344, 0.1115701976]
2     [-0.4324970109, 0.9016317304, -0.0025609041]
3    [-0.4852052527, -0.8620147316, -0.1466508277]
4       [0.0000000000, 0.0000000000, 0.0000000000]
Name: magmoms, dtype: object

In [9]:
print("kpoints =",vasp.structure.kpoints)
print("\nlattice vectors =\n",vasp.structure.lattice_vectors)
print("\nspecies =",vasp.structure.species)
print("\nelements =",vasp.structure.elements)

kpoints = 12 12 12

lattice vectors =
 [[3.95 0.   0.  ]
 [0.   3.95 0.  ]
 [0.   0.   3.95]]

species = ['Ga', 'Mn', 'N']

elements = ['Ga', 'Mn', 'Mn', 'Mn', 'N']
